In [ ]:
import cartopy
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
from cartopy.util import add_cyclic_point

In [ ]:
file_in = 'precip.mon.mean.enh.nc'

In [ ]:
with xr.open_dataset(file_in) as dset:
    print(dset)

In [ ]:
# plot function

def plot_map(data_array):

    fig = plt.figure(figsize=(10, 10))

    ax = plt.axes(projection=cartopy.crs.PlateCarree())

    plt.contourf(
        data_array.lon,
        data_array.lat,
        data_array,
        20,
        transform=cartopy.crs.PlateCarree(),
        cmap=plt.cm.RdYlBu_r
    )

    gl = ax.gridlines(
        crs=cartopy.crs.PlateCarree(),
        draw_labels=True,
        linewidth=0.01
    )

    gl.top_labels = False
    gl.right_labels = False

    # contorno dos estados
    states = cartopy.feature.NaturalEarthFeature(
        category='cultural',
        scale='50m', facecolor='none',
        name='admin_1_states_provinces_shp'
    )
    ax.add_feature(states, edgecolor='k')

    # contorno dos países
    countries = cartopy.feature.NaturalEarthFeature(
        category='cultural',
        scale='50m', facecolor='none',
        name='admin_0_countries'
    )
    ax.add_feature(countries, edgecolor='k')

    plt.colorbar(shrink=0.9, orientation='horizontal')

    plt.show()

In [ ]:
plot_map(dset['precip'].sel(time='2020-03-01'))

In [ ]:
dset['precip'].shape  # longitudes antes do add_cyclic_point

In [ ]:
# add_cyclic_point

lon = dset.coords['lon']

lon_idx = dset['precip'].dims.index('lon')

arr_data, arr_lon = add_cyclic_point(
    dset['precip'].values, coord=lon, axis=lon_idx
)

In [ ]:
arr_data.shape  # mais 1 ponto de grade na longitude!

In [ ]:
arr_lon

In [ ]:
new_dset = xr.Dataset(
    data_vars={
        'precip': (('time', 'lat', 'lon'), arr_data)
    },
    coords={
        'time': dset.time.values,
        'lat': dset.lat.values,
        'lon': arr_lon
    }
)

In [ ]:
plot_map(new_dset['precip'].sel(time='2020-03-01'))

### Corrigir longitudes e latitudes

In [ ]:
# Voltamos para o dset original!

In [ ]:
dset

In [ ]:
# inverter latitudes: 90:-90 para -90:90
dset = dset.reindex(lat=dset['lat'][::-1])

In [ ]:
dset

In [ ]:
dset.lat

In [ ]:
dset['lon_aux'] = xr.where(
    dset['lon'] > 180,
    dset['lon'] - 360,
    dset['lon']
)

dset = (
    dset
        .swap_dims({'lon': 'lon_aux'})
        .sel(**{'lon_aux': sorted(dset.lon_aux)})
        .drop('lon')
)

dset = dset.rename({'lon_aux': 'lon'})

In [ ]:
dset.lon

In [ ]:
dset

In [ ]:
plot_map(dset['precip'].sel(time='2020-03-01'))